## Imports

In [ ]:
%run base.ipynb
%load_ext autoreload
%autoreload 2

import pickle
from pprint import pprint
import tqdm.notebook as tqdm

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import tabulate

# import signature policy and q-learning algorithm
import qlearning_method as qlearning
from qlearning_policies import SigQFunction
import utils

If results and plots from this notebook should be saved set `save=True` and select an identifiying prefix used for file names when saving results and plots, e.g. current date with a letter or number.

In [ ]:
save_flag = False
file_prefix = '20240903_A'

if save_flag:
    # check if files with selected prefix exist
    from pathlib import Path
    if len(list(Path('../results').glob(file_prefix + '*'))) + \
        len(list(Path('../figures').glob(file_prefix + '*'))) != 0:
        file_prefix = input('Files with the chosen prefix already exist.\nPlease enter new prefix.')
    print('Results will be save with chosen prefix:', file_prefix)
else:
    print('Results will not be saved.')

## Environment

In [ ]:
steps = 200
env = gym.make('MountainCar-v0', max_episode_steps=steps)

## Training with different base-points

We want to perform training runs for base-point configurations at the beginning of an episode. Since at $t=0$, there is only only one observation tuple available to the agent, in the Mountain Car environment considered here this is provided as $o_0 = (p_0, 1)$, with $p_0$ the initial position of the car uniformly sampled from $[-0.6, -0.4]$. 

We investigate the approach of appending a base-point $o_{-1} = (p_{-1},1)$ to the history prior to $o_0$ and compute the signature based on the two samples $(o_1, o_0)$. We consider different positions for the base-point position $p_{-1}$. Note that with the use of a fixed basepoint for all observation histories, the signature is not translation invariant.

Additionally, we investigate the approach of using no base-point and simply setting the signature components to $0$ at $t=0$. The first non-zero values for the signature components are computed at $t=1$. Here, the signature of the observation history is translation invariant.

Specify the base-point positions, the configuration of the Signature-Q-Function approximation and the training algorithm hyper-parameters.

In [ ]:
basepoint_positions = [-0.65, -0.50, -0.35, None] # None corresponds to no basepoint
training_runs = 4

# signature config, basepoint will be added below
sigq_params = dict(
    in_channels = 2,
    out_dimension = 3,
    sig_depth = 5,
    initial_bias = -0.1
)

# training config
training_params = dict(
    episodes = 4000,
    discount=0.99,
    learning_rate = 0.005,
    learning_rate_decay = dict(mode=None), 
    epsilon = 0.3,
    epsilon_decay = dict(mode='linear', end_value=0.05, epochs=2000),
    decay_increment = 'success',
    batch_size=1,
)

For each truncation order, we perform four training runs with all other hyper-parameters held fixed. 

In [ ]:
train_res_dict = {}

for bp_position in basepoint_positions:
    basepoint = [bp_position, 1.] if bp_position is not None else None
    bp_position_results = []

    # run training algorithm loop
    pbar = tqdm.trange(training_runs)
    for run in pbar:
        pbar.set_description("Basepoint position: {}  |  Training runs".format(bp_position, run))
        sigQfunction = SigQFunction(initial_basepoint=basepoint, **sigq_params)
        run_results = qlearning.train(env, sigQfunction, **training_params) 
        bp_position_results.append(run_results)      
    
    train_res_dict[bp_position] = bp_position_results 

### Plot training results

#### Single run
Plot the results for a specific basepoint configuration and run. For each episode over the specified run, the following results are plotted:
- the reward obtained, 
- the total loss occured, 
- the car's end position, 
- the steps before reaching the goal or terminating.

In [ ]:
bp_position = -0.5 # -0.65, -0.5, -0.35, None
run_id = 2

basepoint = [bp_position, 1.] if bp_position is not None else None

print("Basepoint position {} in training run {}: {} Successes".format(
    bp_position, run_id, sum(np.array(train_res_dict[bp_position][run_id][2])>=0.5)
))
utils.plot_results(train_res_dict[bp_position], run=run_id, ma_window=100,
                   title="Results for basepoint {}, run {}".format(basepoint, run_id))

#### Averaged over all runs for specific basepoint

For a fixed basepoint configuration, we average the results over all training runs and calculate the mean and standard deviation of
- reward per episode
- loss per episode
- end position per episode
- steps until termination per episode

and plot the average results.

In [ ]:
bp_position = -0.5 # -0.65, -0.5, -0.35, None

# select reward, loss, end position, steps for array
results_array = np.array(
    [ train_res_dict[bp_position][run][0:4] for run in range(len(train_res_dict[bp_position])) ], 
    ndmin=3
)
training_results_means = results_array.mean(axis=0)
training_results_stds = results_array.std(axis=0)

utils.plot_mean_results(training_results_means, training_results_stds, 
                        title='Results averaged over training runs')

#### First Q-values

We check whether the Q-values at the beginning of the episode for a specified basepoint configuration converge towards the same value as the overall reward obtained over the episode. This is an indication, that the algorithm learns the true optimal Q-values of the problem. Unfortunately this is not the case here.

The first Q-values are plotted in two ways:
- The actual first Q-values saved during training for each episode and calculated for the respective observation $o_0$ encountered in each episode.
- The first Q-values for a fixed observation at $t=0$, for example $o_0 = (-0.5, 1.)$, calculated with the intermediate Q-functions saved during training each 10 episodes.

In [ ]:
# choose basepoint position, starting position and run id
bp_position = -0.5 # -0.65, -0.5, -0.35, None
start_position = -0.5
run_id = -1

basepoint = [bp_position, 1.] if bp_position is not None else None
sigqfunction = SigQFunction(initial_basepoint=basepoint, **sigq_params)

# plot with intermediate Q-function approximations
utils.plot_first_Q_values(train_res_dict[bp_position], run_id, intermediate_qfunctions=True,
                          window=(0, training_params['episodes']), sigq_container=sigqfunction,
                          start_position=start_position, show=True)

#### First observation value
Additionally we take a look at the value of the history at time $t=0$ for the start position $p_0 = -0.5$, which corresponds to the middle of the interval $[-0.6, -0.4]$ in which the car is placed randomly at the start of the episode. The value is calculated as the Q-values averaged over actions and is given by
$$
    V_0(\hat{h}_0) = \frac{1}{3}\sum_{i=0}^2 Q(\hat{h}_0, a_i),   
$$
It gives the value over an episode following a greedy policy based on the current approximate Q-function $Q$. If $V_0(\hat{h}_0)$ converges towards the average reward over an episode, this indicates that the algorithm has converged to the true Q-values (at least for the Q-values at the start of the episode). Unfortunatly this is not the case here, since the algorithm over estimates Q-values.

The plot displays the mean and standard deviation of $V_0(\hat{h}_0)$ calculated over all performed training runs. 

In [ ]:
# choose basepoint position 
bp_position = -0.5 # -0.65, -0.5, -0.35, None

# create array containing saved Q-values for all runs
first_observations_array = np.array(
    [train_res_dict[bp_position][run][-3] for run in range(training_runs)]
)
value_means = first_observations_array.mean(axis=0)
values_stds = first_observations_array.std(axis=0)

# plot and save
utils.plot_first_obs_value(value_means, values_stds)

## Test Q-functions

We test the final Q-approximations for each basepoint configuration and from each training run over a number of episodes and report statistics. 

Checkpoints of the `signatureQFunction` approximations during training was saved every 10 episodes and after the last training episode in each training run. The final learned approximation corresponds to the last checkpoint, given as `state_dict` of the `SigQFunction` class instance at the checkpoint time. Other approximation from training may be tested by selecting the appropriate checkpoint.

In [ ]:
# choose number of test episodes and Q-function checkpoint
test_episodes = 500
sigq_checkpoint_id = -1

test_res_dict = {}
for bp_position in basepoint_positions:
    basepoint = [bp_position, 1.] if bp_position is not None else None
    sigqfunction = SigQFunction(initial_basepoint=basepoint, **sigq_params)
    training_results = train_res_dict[bp_position]
    test_results = []

    pbar = tqdm.trange(training_runs)
    for run in pbar:
        pbar.set_description('Basepoint position:  {}  |  Test runs'.format(bp_position))
        # load last Sig-Q approximation parameters
        sigqfunction.load_state_dict(training_results[run][-1][sigq_checkpoint_id])
        results = qlearning.test_multiple_episodes(env, sigqfunction, 
                                                   test_episodes, epsilon=0.0)
        test_results.append(results)
    
    test_res_dict[bp_position] = test_results

### Test result statistics

We display statistics as one table for each truncation order. This output format is not the best since we want to compare performance over truncation orders, however it will do to transfer it into a different format in the thesis document.

In [ ]:
test_stats_dict = {}
cols = ['Mean\nreward', 'Std Dev\nreward', 'Successes', 'Mean\nsteps', 'Std Dev\nsteps', 'Min\nsteps', 'Max\nsteps', 'Mean first\nobs value']
rows = ['Run {}'.format(i) for i in range(training_runs)]

for bp_position in basepoint_positions:
    bp_position_stats = []
    for run in test_res_dict[bp_position]:
        run_array = np.array(run[0:5])
        run_stats = []
        run_stats.append(run_array[0].mean()) # mean reward
        run_stats.append(run_array[0].std()) # std reward
        run_stats.append(100*run_array[1].sum()/test_episodes) # percentage successes
        run_stats.append(run_array[2].mean()) # mean episode length
        run_stats.append(run_array[2].std()) # std episode length
        run_stats.append((int(run_array[2].min()))) # min episode steps
        run_stats.append(int(run_array[2].max())) # max episode steps
        run_stats.append(run_array[4].mean()) # first observation value
        
        bp_position_stats.append(run_stats)

    test_stats_dict[bp_position] = bp_position_stats
    print(f'\n\nTest statistic for basepoint position {bp_position}:\n')
    print(tabulate.tabulate(bp_position_stats, headers=cols, showindex=rows,floatfmt='.4f'))

### Scatterplots

Starting position vs. steps until termination for all basepoint configurations.

In [ ]:
run_id = 0

# start position vs. number of steps
for bp_position in basepoint_positions:
    plt.figure(figsize=(5.5, 4.125))        
    plt.scatter(test_res_dict[bp_position][run_id][3], 
                test_res_dict[bp_position][run_id][2], 
                marker='x')
    plt.xlabel("Start position", fontsize=11)
    plt.ylabel("Number of steps", fontsize=11)
    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.title(' ', fontsize=12) # some space above plot
    plt.tight_layout()
    file_path = '../figures/{}_start_vs_steps_bp_position_{}.png'.format(file_prefix, bp_position)
    if save_flag:
        plt.savefig(file_path)
    plt.show()

Starting position vs. reward for all basepoint configurations

In [ ]:
run_id = 0

# start position vs. rewards
for bp_position in basepoint_positions:
    plt.figure(figsize=(5.5, 4.125))        
    plt.scatter(test_res_dict[bp_position][run_id][3], 
                test_res_dict[bp_position][run_id][0], 
                marker='x')
    plt.xlabel("Start position", fontsize=11)
    plt.ylabel("Number of steps", fontsize=11)
    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.title(' ', fontsize=12) # some space above plot
    plt.tight_layout()
    file_path = '../figures/{}_start_vs_steps_bp_position_{}.png'.format(file_prefix, bp_position)
    if save_flag:
        plt.savefig(file_path)
    plt.show()

To gain an even more qualitative insight into the policy derived from the learned Q-function approximation, we may plot the observation trajectory for a specific truncation order, run and episode.

In [ ]:
bp_position, run_id, episode_id = None, 0, 70
# observation history, (position, time)
plt.plot(test_res_dict[bp_position][run_id][-1][episode_id])
plt.show()

## Save results

In [ ]:
if save_flag:
    # create file name and write results
    file_path = '../results/' + file_prefix + '_basepoint_results.pkl'
    data_to_save = dict(basepoint_positions=basepoint_positions,
                        sigq_params = sigq_params, 
                        training_params=training_params,
                        train_res_dict=train_res_dict, 
                        test_res_dict=test_res_dict)
    with open(file_path, 'wb') as f:  # open text file
        pickle.dump(data_to_save, f) # serialize the dict
    print(
        'Training, test results, parameter configuration saved under\n\"{}\".'.format(
            file_path
        ))

## Load results

Instead of performing new training and testing runs, prior saved training and testing results, together with the parameter configuration used in training, may be loaded. Saved results can be found in `../results/` and are identified by a distinct `file_prefix`, e.g. the date the runs were performed, in the format `YYYYMMDD`, together with an upper case letter which enumerates the result saved on the same date. To load results select the respective `file_prefix` of the results to be loaded and set `execute_cell_flag = True`.

The following objects are loaded:
- `truncation_orders` - list containing the tested signature truncation orders
- `sigq_params` - dict containing the signature-Q-function parameters
- `training_params` - dict containing the training algorithm parameters
- `train_res_dict` - dict, contains training run results for each truncation order
- `test_res_dict` - dict, contains test run results for each truncation order
– `training_runs`- int, number of performed training runs per truncation order

**Note:** To display results after loading them, the respective cells above this section need to be executed. 

In [83]:
execute_cell_flag = False

load_file_prefix = '20240903_A'
file_path = '../results/' + load_file_prefix + '_basepoint_results.pkl'

if execute_cell_flag:
    # set date and id of saved results to load
    with open(file_path, 'rb') as f:
        loaded_data_dict = pickle.load(f)
    f.close()
    
    basepoint_positions = loaded_data_dict['basepoint_positions']
    sigq_params = loaded_data_dict['sigq_params']
    training_params = loaded_data_dict['training_params']
    train_res_dict = loaded_data_dict['train_res_dict']
    test_res_dict = loaded_data_dict['test_res_dict']
    training_runs = len(list(train_res_dict.values())[0])

    print('Training, test results, parameter configuration loaded from: \n \"{}\"\n'.format(file_path))
    print('Number of training runs: {}.\nWith parameters:\n'.format(training_runs))
    pprint({key:loaded_data_dict[key] for key in loaded_data_dict if key not in ('train_res_dict', 'test_res_dict')})

Training, test results, parameter configuration loaded from: 
 "../results/20240903_A_basepoint_results.pkl"

Number of training runs: 4.
With parameters:

{'basepoint_positions': [-0.65, -0.5, -0.35, None],
 'sigq_params': {'in_channels': 2,
                 'initial_bias': 0.1,
                 'out_dimension': 3,
                 'sig_depth': 5},
 'training_params': {'batch_size': 1,
                     'decay_increment': 'success',
                     'discount': 0.99,
                     'episodes': 4000,
                     'epsilon': 0.3,
                     'epsilon_decay': {'end_value': 0.05,
                                       'epochs': 2000,
                                       'mode': 'linear'},
                     'learning_rate': 0.005,
                     'learning_rate_decay': {'mode': None}}}
